In [1]:
# get the status of the bridges
import tweepy 
import time
import csv
from datetime import datetime
from datetime import timedelta
from config import *
import bridge_stat
import pandas as pd

In [2]:
tweets = bridge_stat.get_tweets('SDOTBridges',export_all=False)

In [42]:
df = bridge_stat.bridge_status(tweets)
# df = tweets

In [43]:
# get a stored bridge status file
# df = pd.read_csv("../data/bridge/bridge_status.csv")
df = df.sort('local_date')
# note that "created_at" may be chornologically accurate,
# but the actual time is wrong

# df['timeobj'] = pd.to_datetime(df['timeobj'])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [44]:
df.tail()

,bridge,event,text,created_at,local_date
4,Fremont,closed,The Fremont Bridge has closed to traffic -\n ...,2016-09-07 22:08:03,2016-09-07 15:08:03
3,Fremont,open,The Fremont Bridge has reopened to traffic -\n...,2016-09-07 22:10:15,2016-09-07 15:10:15
2,Lower Spokane St,closed,The Lower Spokane St Bridge has closed to\n t...,2016-09-07 23:46:51,2016-09-07 16:46:51
1,Lower Spokane St,open,The Lower Spokane St Bridge has reopened to\n ...,2016-09-08 00:04:20,2016-09-07 17:04:20
0,Fremont,closed,The Fremont Bridge has closed to traffic -\n ...,2016-09-08 01:15:54,2016-09-07 18:15:54


In [45]:
# Get a list of bridges that opened but have not closed

# All closed bridges should have an open even followed by a later closed event

# Check all the close events
close_events = df[df['event'] == 'closed'].drop_duplicates(subset='bridge', keep='last')
open_events = df[df['event'] == 'open'].drop_duplicates(subset='bridge', keep='last')

In [46]:
# get only the most recent open and close events
df = pd.merge(close_events,open_events,on='bridge',suffixes=['_close','_open'])

In [47]:
# fake some data for testing - set the last close time to a later date
# df['local_date_open'].iloc[0] = df['local_date_close'].iloc[0].replace(hour=12)

In [51]:
# current_closures = df[(df['_open'] != df['date_close']) | (df['created_at_open'] < df['crated_at_close'])]
current_closures = df[df['local_date_open'] < df['local_date_close']]

In [52]:
current_closures

,bridge,event_close,text_close,created_at_close,local_date_close,event_open,text_open,created_at_open,local_date_open
6,Fremont,closed,The Fremont Bridge has closed to traffic -\n ...,2016-09-08 01:15:54,2016-09-07 18:15:54,open,The Fremont Bridge has reopened to traffic -\n...,2016-09-07 22:10:15,2016-09-07 15:10:15


In [10]:
# Map each closure
# provide an expected window of re-opening based on historical closure data
# show time closed, bell curve of likely re-opening

In [11]:
# Create some leaflet thing here
# try folium: https://github.com/python-visualization/folium

In [54]:
bridge_list = ['Ballard', 'Fremont', '1st Ave S', 'Montlake', 'Lower Spokane St', 'South Park', 'University']

In [61]:
import folium
import vincent
import json
vincent.core.initialize_notebook()

In [78]:
bridges = {'Lower Spokane St': [47.570950, -122.348324],
          'South Park': [47.529252, -122.314316],
          'Fremont': [47.647857, -122.349867],
          'Ballard': [47.659280, -122.376163],
          'Montlake': [47.647296, -122.304577],
          'University': [47.652995, -122.320194],
          '1st Ave S': [47.542049, -122.334541]}

In [83]:
# systematic pop up for all bridges, based on status
map_osm = folium.Map(location=[47.5836, -122.3750], zoom_start=11, tiles='Stamen Toner')

for bridge, coord in bridges.iteritems():
    color='green'
    print bridge
    if bridge in current_closures['bridge'].values:
        color='red'
    icon = folium.Icon(color=color, icon="ok")
    marker = folium.Marker(location=coord,icon=icon)

    map_osm.add_children(marker);

map_osm.save('osm.html')

Lower Spokane St
Ballard
University
Fremont
Montlake
1st Ave S
South Park


In [84]:
map_osm

In [16]:
# Test with a single marker and a popup graph
map_osm = folium.Map(location=[47.5836, -122.3750], zoom_start=12, tiles='Stamen Toner')
# folium.Marker([47.570950, -122.348324], popup='Spokane Street').add_to(map_osm)
# folium.Marker([47.529252, -122.314316], popup='South Park').add_to(map_osm)

# get fancy 
# see e.g., http://nbviewer.jupyter.org/github/ocefpaf/folium_notebooks/blob/master/test_png_mpld3_vega_popup.ipynb
# show some fake data
resolution, width, height = 50, 7, 3

vis = vincent.Line(df['test'], width=width*resolution, height=height*resolution)
vis.legend(title='Spokane');
vega = folium.Vega(json.loads(vis.to_json()), width="30%", height="30%")
popup = folium.Popup(max_width=vis.width+75).add_child(vega)
icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[47.570950, -122.348324], popup=popup, icon=icon)

map_osm.add_children(marker);

map_osm.save('osm.html')

In [17]:
map_osm

In [ ]:
# pop window should show time elapsed since road closure
# maybe as a progress bar
# or estimated time remaining open based on past records & ship data
# 

[47.639252, -122.384316]
[47.57095, -122.348324]
[47.529252, -122.314316]


In [63]:
map_osm